#SPY Spreadsheet

In [29]:
%matplotlib inline

Use future imports for python 3.0 forward compatibility

In [30]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

Other imports

In [31]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import pinkfish as pf
import itable

# format price data
pd.options.display.float_format = '{:0,.2f}'.format

# Double the DPI, so we are making 2x plots:
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

Some global data

In [32]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [33]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [34]:
ts.tail()

,open,high,low,close,volume,adj_close
date,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [35]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [36]:
ts.head()

,open,high,low,close,volume,adj_close
date,,,,,,
1993-01-29,28.48,28.48,28.33,28.46,1003200,28.46
1993-02-01,28.48,28.66,28.48,28.66,480500,28.66
1993-02-02,28.64,28.74,28.58,28.72,201300,28.72
1993-02-03,28.76,29.04,28.74,29.02,529400,29.02
1993-02-04,29.12,29.20,28.80,29.14,531500,29.14


Add technical indicator: 200 day MA

In [37]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [38]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200
date,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72


Add technical indicator: ATR

In [39]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [40]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr
date,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94


Add technical indicator: 5 day high, and 5 day low

In [41]:
high5 = pd.rolling_max(ts.high, 5)
low5 = pd.rolling_min(ts.low, 5)
ts['high5'] = high5
ts['low5'] = low5

In [42]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93,208.18,203.88
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86,206.80,203.88
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91,208.47,203.88
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92,208.54,203.88
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94,208.54,203.88


Add technical indicator: RSI, and 2-period cumulative RSI

In [43]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.rolling_sum(ts.rsi2, 2)
ts['c2rsi2'] = c2rsi2

In [44]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93,208.18,203.88,58.84,74.02
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86,206.80,203.88,66.63,125.47
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91,208.47,203.88,95.02,161.65
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92,208.54,203.88,41.38,136.40
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94,208.54,203.88,43.35,84.74


Add technical indicator: Midpoint

In [45]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [46]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93,208.18,203.88,58.84,74.02,204.83
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86,206.80,203.88,66.63,125.47,205.88
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91,208.47,203.88,95.02,161.65,207.56
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92,208.54,203.88,41.38,136.40,207.52
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94,208.54,203.88,43.35,84.74,206.43


Add technical indicator: SMA10 of midpoint

In [47]:
sma10mp = pd.rolling_mean(ts.mp, 10)
ts['sma10mp'] = sma10mp

In [48]:
ts.head(10)

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,28.48,28.48,28.33,28.46,1003200,28.46,nan,nan,nan,nan,nan,nan,28.40,nan
1993-02-01,28.48,28.66,28.48,28.66,480500,28.66,nan,nan,nan,nan,nan,nan,28.57,nan
1993-02-02,28.64,28.74,28.58,28.72,201300,28.72,nan,nan,nan,nan,100.00,nan,28.66,nan
1993-02-03,28.76,29.04,28.74,29.02,529400,29.02,nan,nan,nan,nan,100.00,200.00,28.89,nan
1993-02-04,29.12,29.20,28.80,29.14,531500,29.14,nan,nan,29.20,28.33,100.00,200.00,29.00,nan
1993-02-05,29.12,29.18,28.96,29.12,492100,29.12,nan,nan,29.20,28.48,89.32,189.32,29.07,nan
1993-02-08,29.12,29.22,29.08,29.12,596100,29.12,nan,nan,29.22,28.58,89.32,178.64,29.15,nan
1993-02-09,29.02,29.02,28.86,28.92,122100,28.92,nan,nan,29.22,28.74,16.96,106.28,28.94,nan
1993-02-10,28.92,28.98,28.84,28.96,379600,28.96,nan,nan,29.22,28.80,37.28,54.25,28.91,nan


Add technical indicator: Standard Deviation

In [49]:
sd = pd.rolling_std(ts.mp, 10)
ts['sd'] = sd

In [50]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93,208.18,203.88,58.84,74.02,204.83,206.89,1.62
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86,206.80,203.88,66.63,125.47,205.88,206.67,1.59
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91,208.47,203.88,95.02,161.65,207.56,206.53,1.42
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92,208.54,203.88,41.38,136.40,207.52,206.39,1.21
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94,208.54,203.88,43.35,84.74,206.43,206.20,1.00


In [51]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [52]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2016-05-06,204.06,205.77,203.88,205.72,83784900,205.72,199.69,1.93,208.18,203.88,58.84,74.02,204.83,206.89,1.62,210.13,203.65
2016-05-09,205.57,206.40,205.36,205.89,68187900,205.89,199.68,1.86,206.80,203.88,66.63,125.47,205.88,206.67,1.59,209.85,203.49
2016-05-10,206.72,208.47,206.64,208.45,74236100,208.45,199.70,1.91,208.47,203.88,95.02,161.65,207.56,206.53,1.42,209.37,203.69
2016-05-11,207.91,208.54,206.50,206.50,77749700,206.50,199.72,1.92,208.54,203.88,41.38,136.40,207.52,206.39,1.21,208.81,203.97
2016-05-12,207.29,207.49,205.37,206.56,88260800,206.56,199.72,1.94,208.54,203.88,43.35,84.74,206.43,206.20,1.00,208.19,204.20


Select a smaller time from for use with itable

In [53]:
df = ts['2015-01-01':]

In [54]:
df.head()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2015-01-02,201.17,201.66,199.03,200.24,121465900,200.24,189.16,2.23,203.69,199.03,6.95,14.45,200.34,202.16,0.94,204.05,200.27
2015-01-05,199.02,199.21,196.27,196.63,169632600,196.63,189.25,2.36,203.69,196.27,1.19,8.14,197.74,201.73,1.69,205.10,198.35
2015-01-06,196.99,197.60,193.84,194.78,209151400,194.78,189.33,2.46,203.11,193.84,0.64,1.83,195.72,201.17,2.55,206.27,196.08
2015-01-07,196.34,197.60,195.81,197.20,125346700,197.20,189.42,2.48,202.93,193.84,54.91,55.56,196.71,200.67,2.90,206.46,194.87
2015-01-08,198.86,200.96,198.84,200.70,147217800,200.70,189.53,2.57,201.66,193.84,82.49,137.41,199.90,200.40,2.83,206.05,194.75


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [55]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme="theme1"), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc("high5")    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc("low5")
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [57]:
pt

,OPEN,HIGH,LOW,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2015/01/02,201.17,201.66,199.03,200.24,121465900,200.24,189.16,2.23,203.69,199.03,6.95,14.45,200.34,202.16,0.94,204.05,200.27
2015/01/05,199.02,199.21,196.27,196.63,169632600,196.63,189.25,2.36,203.69,196.27,1.19,8.14,197.74,201.73,1.69,205.10,198.35
2015/01/06,196.99,197.60,193.84,194.78,209151400,194.78,189.33,2.46,203.11,193.84,0.64,1.83,195.72,201.17,2.55,206.27,196.08
2015/01/07,196.34,197.60,195.81,197.20,125346700,197.20,189.42,2.48,202.93,193.84,54.91,55.56,196.71,200.67,2.90,206.46,194.87
2015/01/08,198.86,200.96,198.84,200.70,147217800,200.70,189.53,2.57,201.66,193.84,82.49,137.41,199.90,200.40,2.83,206.05,194.75
2015/01/09,201.19,201.21,198.37,199.09,158567300,199.09,189.64,2.59,201.21,193.84,52.80,135.29,199.79,200.10,2.70,205.50,194.70
2015/01/12,199.25,199.43,196.82,197.53,144396100,197.53,189.74,2.59,201.21,193.84,31.10,83.90,198.13,199.59,2.51,204.61,194.56
2015/01/13,198.97,200.29,195.45,196.98,214553300,196.98,189.84,2.75,201.21,195.45,24.05,55.15,197.87,199.05,2.19,203.42,194.67
2015/01/14,194.61,196.02,193.56,195.79,192991100,195.79,189.92,2.80,201.21,193.56,12.21,36.26,194.79,198.26,2.15,202.56,193.95
2015/01/15,196.54,196.91,193.86,194.00,176613900,194.00,189.98,2.82,201.21,193.56,4.91,17.13,195.38,197.64,1.97,201.59,193.69
